In [1]:
import torch
print(torch.cuda.is_available())  
print(torch.cuda.current_device())  
print(torch.cuda.get_device_name(torch.cuda.current_device())) 

True
0
NVIDIA GeForce RTX 4060 Laptop GPU


In [2]:
import sys,os
__script_path=os.path.abspath(globals().get('__file__','.'))
__script_dir = os.path.dirname(__script_path)
root_dir = os.path.abspath(f'{__script_dir}/../../../../..')
print(root_dir)
for lib in [root_dir][::-1]:
    if lib in sys.path:
        sys.path.remove(lib)
    sys.path.insert(0,lib)
from config.config import *
# from libs.common import *

c:\Users\Admin\Data\ads_fraud_detection
c:/Users/Admin/Data/ads_fraud_detection


In [3]:
save_dir=f"{exps_dir}/exp2/exp_smote"
if os.path.exists(save_dir) == False: 
  os.makedirs(save_dir, exist_ok=True)



test_size=0.33
seed=42

In [4]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA is available. Using GPU.")
else:
    device = torch.device("cpu")
    print("CUDA is not available. Using CPU.")

CUDA is available. Using GPU.


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from pytorch_tabnet.pretraining import TabNetPretrainer
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import numpy as np


In [6]:
x_train=pd.read_excel(f'{save_dir}/x_train.xlsx')
y_train=pd.read_excel(f'{save_dir}/y_train.xlsx')
x_test=pd.read_excel(f'{save_dir}/x_test.xlsx')
y_test=pd.read_excel(f'{save_dir}/y_test.xlsx')
class_weights_dict=dict(np.load(f'{save_dir}/class_weights_dict.npz',allow_pickle=True))['class_weights_dict']
class_weights_dict = {key: value for key, value in class_weights_dict.item().items()}
class_weights_dict

{0: 0.6378163041991519, 1: 2.3140088827134457}

In [7]:
X_train = torch.tensor(x_train.values, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.long)
X_test = torch.tensor(x_test.values, dtype=torch.float32)
y_test = torch.tensor(y_test.values, dtype=torch.long)
num_classes = 2  # Assuming binary classification
y_train = F.one_hot(y_train, num_classes=num_classes)
y_test = F.one_hot(y_test, num_classes=num_classes)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)


torch.Size([19430, 33]) torch.Size([19430, 1, 2]) torch.Size([5089, 33]) torch.Size([5089, 1, 2])


In [8]:
from torch.utils.data import TensorDataset, DataLoader
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

In [9]:
tabnet_params = {
    "n_d": 512,
    "n_a": 512,
    "n_steps": 3,
    "n_shared": 2,
    "n_independent": 2,
    "gamma": 1.3,
    "epsilon": 1e-15,
    "momentum": 0.98,
    "mask_type": "sparsemax",
    "lambda_sparse": 1e-3,
    "device_name": "cuda" if torch.cuda.is_available() else "cpu"
}


unsupervised_model = TabNetPretrainer(
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=2e-2),
    **tabnet_params
)
 

unsupervised_model.fit(
    X_train,
    eval_set=[X_test],  
    pretraining_ratio=0.8,
    max_epochs=101,
    patience=10,
    batch_size=1024,
    virtual_batch_size=128,
    num_workers=0,
    drop_last=False
)

c:\Users\Admin\miniconda3\envs\pytorch\lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 1941.51756| val_0_unsup_loss_numpy: 21.037090301513672|  0:00:01s
epoch 1  | loss: 22.06558| val_0_unsup_loss_numpy: 4.085599899291992|  0:00:03s
epoch 2  | loss: 8.62228 | val_0_unsup_loss_numpy: 6.7566399574279785|  0:00:04s
epoch 3  | loss: 8.90729 | val_0_unsup_loss_numpy: 2.1685800552368164|  0:00:05s
epoch 4  | loss: 4.31033 | val_0_unsup_loss_numpy: 88.85392761230469|  0:00:07s
epoch 5  | loss: 6.68755 | val_0_unsup_loss_numpy: 1.4134299755096436|  0:00:08s
epoch 6  | loss: 4.51434 | val_0_unsup_loss_numpy: 1.9488600492477417|  0:00:10s
epoch 7  | loss: 5.04796 | val_0_unsup_loss_numpy: 1.7437700033187866|  0:00:11s
epoch 8  | loss: 2.8203  | val_0_unsup_loss_numpy: 6.641610145568848|  0:00:13s
epoch 9  | loss: 2.32682 | val_0_unsup_loss_numpy: 1.6450400352478027|  0:00:14s
epoch 10 | loss: 2.08004 | val_0_unsup_loss_numpy: 2.615489959716797|  0:00:16s
epoch 11 | loss: 2.23871 | val_0_unsup_loss_numpy: 1.3475799560546875|  0:00:17s
epoch 12 | loss: 1.88427 | val

c:\Users\Admin\miniconda3\envs\pytorch\lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [10]:
# Truy cập vào mô hình TabNet bên trong
from torchinfo import summary

tabnet_model = unsupervised_model.network.to(device)

summary(tabnet_model, input_size=X_train.shape) 

Layer (type:depth-idx)                                       Output Shape              Param #
TabNetPretraining                                            [19430, 33]               --
├─EmbeddingGenerator: 1-1                                    [19430, 33]               --
├─TabNetEncoder: 1-2                                         [19430, 512]              --
│    └─BatchNorm1d: 2-1                                      [19430, 33]               66
│    └─FeatTransformer: 2-2                                  [19430, 1024]             4,202,496
│    │    └─GLU_Block: 3-1                                   [19430, 1024]             2,172,928
│    └─ModuleList: 2-12                                      --                        (recursive)
│    │    └─FeatTransformer: 3-17                            --                        (recursive)
│    └─FeatTransformer: 2-6                                  --                        (recursive)
│    │    └─GLU_Block: 3-5                            

In [11]:
encoder = tabnet_model.encoder

print("\nEncoder Summary:")
print(encoder)




Encoder Summary:
TabNetEncoder(
  (initial_bn): BatchNorm1d(33, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
  (initial_splitter): FeatTransformer(
    (shared): GLU_Block(
      (shared_layers): ModuleList(
        (0): Linear(in_features=33, out_features=2048, bias=False)
        (1): Linear(in_features=1024, out_features=2048, bias=False)
      )
      (glu_layers): ModuleList(
        (0): GLU_Layer(
          (fc): Linear(in_features=33, out_features=2048, bias=False)
          (bn): GBN(
            (bn): BatchNorm1d(2048, eps=1e-05, momentum=0.98, affine=True, track_running_stats=True)
          )
        )
        (1): GLU_Layer(
          (fc): Linear(in_features=1024, out_features=2048, bias=False)
          (bn): GBN(
            (bn): BatchNorm1d(2048, eps=1e-05, momentum=0.98, affine=True, track_running_stats=True)
          )
        )
      )
    )
    (specifics): GLU_Block(
      (glu_layers): ModuleList(
        (0-1): 2 x GLU_Layer(
          (fc

In [12]:
decoder = tabnet_model.decoder

print("\nDecoder Summary:")
print(decoder)


Decoder Summary:
TabNetDecoder(
  (feat_transformers): ModuleList(
    (0-2): 3 x FeatTransformer(
      (shared): GLU_Block(
        (shared_layers): ModuleList(
          (0): Linear(in_features=512, out_features=1024, bias=False)
        )
        (glu_layers): ModuleList(
          (0): GLU_Layer(
            (fc): Linear(in_features=512, out_features=1024, bias=False)
            (bn): GBN(
              (bn): BatchNorm1d(1024, eps=1e-05, momentum=0.98, affine=True, track_running_stats=True)
            )
          )
        )
      )
      (specifics): GLU_Block(
        (glu_layers): ModuleList(
          (0): GLU_Layer(
            (fc): Linear(in_features=512, out_features=1024, bias=False)
            (bn): GBN(
              (bn): BatchNorm1d(1024, eps=1e-05, momentum=0.98, affine=True, track_running_stats=True)
            )
          )
        )
      )
    )
  )
  (reconstruction_layer): Linear(in_features=512, out_features=33, bias=False)
)


In [13]:
sample_input = torch.tensor(X_train[:5]).to(device)  

try:
    result = tabnet_model.encoder(sample_input)
    if isinstance(result, tuple):
        print(f'TabNet encoder trả về {len(result)} giá trị.')
        for i, res in enumerate(result):
            print(f'Giá trị {i + 1} shape: {res.shape}')
    else:
        print('TabNet encoder chỉ trả về một giá trị.')
        print(f'Giá trị shape: {result.shape}')
except Exception as e:
    print(f'Đã xảy ra lỗi: {e}')

TabNet encoder trả về 2 giá trị.
Đã xảy ra lỗi: 'list' object has no attribute 'shape'


C:\Users\Admin\AppData\Local\Temp\ipykernel_23028\3356785120.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample_input = torch.tensor(X_train[:5]).to(device)


In [14]:
class Sampling(nn.Module):
    def __init__(self, seed=1337):
        super(Sampling, self).__init__()
        self.seed = seed

    def forward(self, inputs):
        z_mean, z_log_var = inputs
        batch = z_mean.size(0)
        dim = z_mean.size(1)
        # print(batch, dim)
        epsilon = torch.randn(batch, dim, generator=torch.Generator().manual_seed(self.seed)).to(device)
        return z_mean + torch.exp(0.5 * z_log_var) * epsilon

In [15]:
# class VAE_Encoder(nn.Module):
#     def __init__(self, latent_dim):
#         super(VAE_Encoder, self).__init__()
#         self.tabnet_encoder = tabnet_model.encoder
#         self.mlp = nn.Sequential(
#             nn.Linear(512, 256),  
#             nn.ReLU(),
#             nn.Linear(256, 128),
#             nn.ReLU(),
#             nn.Linear(128, 96),
#             nn.ReLU(),
#             nn.Linear(96, 96),
#             nn.ReLU(),
#             nn.Linear(96, 32),
#             nn.ReLU(),
#             nn.Linear(32, latent_dim)
#         ).to(device)
#         self.fc_mean = nn.Linear(latent_dim, latent_dim).to(device)
#         self.fc_log_var = nn.Linear(latent_dim, latent_dim).to(device)
#         self.sampling = Sampling().to(device)

#     def forward(self, x):
#         x = x.to(device)
#         steps_output, _ = self.tabnet_encoder(x)
#         encoded = steps_output[-1]
#         # print("Shape of encoded tensor:", encoded.shape)
#         encoded = self.mlp(encoded)
#         z_mean = self.fc_mean(encoded)
#         z_log_var = self.fc_log_var(encoded)
#         z = self.sampling((z_mean, z_log_var))
#         # print(f'Shape of z: {z.shape} - {z_log_var.shape} -{z_log_var.shape}')
#         return z_mean, z_log_var, z


In [ ]:
class VAE_Encoder(nn.Module):
    def __init__(self, latent_dim):
        super(VAE_Encoder, self).__init__()
        # Giữ nguyên TabNet encoder
        self.tabnet_encoder = tabnet_model.encoder

        # Thay thế MLP bằng Conv1D
        self.conv_branch_0 = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool1d(128),  # Giữ cố định kích thước
        )

        self.conv_branch_1 = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool1d(128),
        )

        # Ánh xạ sang latent space
        self.fc_mean_0 = nn.Linear(32 * 128, latent_dim)
        self.fc_log_var_0 = nn.Linear(32 * 128, latent_dim)

        self.fc_mean_1 = nn.Linear(32 * 128, latent_dim)
        self.fc_log_var_1 = nn.Linear(32 * 128, latent_dim)

        self.sampling = Sampling()
        self.latent_dim = latent_dim

    def forward(self, x, labels):
        if labels.dim() == 2 and labels.size(1) == 2:
            labels = torch.argmax(labels, dim=1)
        labels = labels.long().to(x.device)

        # Qua TabNet encoder
        steps_output, _ = self.tabnet_encoder(x)
        encoded = steps_output[-1]  # shape: [batch_size, 512]

        batch_size = encoded.shape[0]
        z_mean = torch.zeros(batch_size, self.latent_dim).to(x.device)
        z_log_var = torch.zeros(batch_size, self.latent_dim).to(x.device)
        z = torch.zeros(batch_size, self.latent_dim).to(x.device)

        idx_0 = (labels == 0)
        idx_1 = (labels == 1)

        # Xử lý lớp 0
        if idx_0.any():
            h0 = encoded[idx_0].unsqueeze(1)  # [B', 1, 512]
            h0 = self.conv_branch_0(h0)       # [B', 32, 128]
            h0 = h0.view(h0.shape[0], -1)     # [B', 32*128]

            mu0 = self.fc_mean_0(h0)
            logvar0 = self.fc_log_var_0(h0)
            z0 = self.sampling((mu0, logvar0))

            z_mean[idx_0] = mu0
            z_log_var[idx_0] = logvar0
            z[idx_0] = z0

        # Xử lý lớp 1
        if idx_1.any():
            h1 = encoded[idx_1].unsqueeze(1)  # [B', 1, 512]
            h1 = self.conv_branch_1(h1)       # [B', 32, 128]
            h1 = h1.view(h1.shape[0], -1)     # [B', 32*128]

            mu1 = self.fc_mean_1(h1)
            logvar1 = self.fc_log_var_1(h1)
            z1 = self.sampling((mu1, logvar1))

            z_mean[idx_1] = mu1
            z_log_var[idx_1] = logvar1
            z[idx_1] = z1

        return z_mean, z_log_var, z

In [ ]:
class VAE_Decoder(nn.Module):
    def __init__(self, latent_dim, encoded_dim=512, output_dim=None):
        super(VAE_Decoder, self).__init__()
        self.encoded_dim = encoded_dim
        self.output_dim = output_dim  # Số lượng class hoặc số feature cần sinh ra

        self.decoder_input = nn.Linear(latent_dim, 32 * encoded_dim)
        self.unflatten = nn.Unflatten(1, (32, encoded_dim))

        self.conv_decoder = nn.Sequential(
            nn.ConvTranspose1d(32, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.ConvTranspose1d(16, 1, kernel_size=3, stride=1, padding=1),
        )

        # Dùng TabNet decoder cuối cùng
        self.tabnet_decoder = tabnet_model.decoder

    def forward(self, z):
        x = self.decoder_input(z)  # [B, 32 * 512]
        x = self.unflatten(x)  # [B, 32, 512]
        x = self.conv_decoder(x)  # [B, 1, 512]
        x = x.squeeze(1)  # [B, 512]

        # Truyền vào TabNet decoder
        x = x.unsqueeze(0)  # [1, B, 512]
        output = self.tabnet_decoder(x)  # [T, B, output_dim]

        # Áp dụng softmax trên dim cuối cùng
        output = F.softmax(output, dim=-1)

        # Flatten lại nếu cần so sánh với ground-truth one-hot
        output = output.view(-1, self.output_dim)

        return output

In [18]:
def check_data_range(tensor, name):
    if not torch.all((tensor >= 0) & (tensor <= 1)):
        print(f"{name} contains values outside the range [0, 1]")
        print(f"{name} min: {tensor.min()}, max: {tensor.max()}")

In [19]:
class VAE_Tabnet_MLPS(nn.Module):
    def __init__(self, encoder, decoder, classifier):
        super(VAE_Tabnet_MLPS, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.classifier = classifier
        self.total_loss_tracker = []
        self.reconstruction_loss_tracker = []
        self.kl_loss_tracker = []
        self.classification_loss_tracker = []
        self.accuracy_tracker = []

    def forward(self, x, labels):
        z_mean, z_log_var, z = self.encoder(x, labels)
        reconstruction = self.decoder(z)
        classification_output = self.classifier(z)
        return reconstruction, z_mean, z_log_var, classification_output

    def train_step(self, data, labels, optimizer):
        labels = labels.squeeze(1) 
        labels = torch.argmax(labels, dim=1).long()
        labels = labels.to(device)
        # print(f"Shape of labels: {labels.shape}")
        optimizer.zero_grad()
        # z_mean, z_log_var, z = self.encoder(data)
        # reconstruction = self.decoder(z)
        reconstruction, z_mean, z_log_var, classification_output = self.forward(data, labels)
        # print('classifi',classification_output.shape)
        # print(check_data_range(data, 'data'))
        # print(check_data_range(reconstruction, 'reconstruction'))
        # reconstruction_loss = torch.mean(
        #     torch.sum(
        #         F.binary_cross_entropy(reconstruction, data, reduction='none'),
        #         dim=1
        #     )
        # )
        reconstruction_loss = torch.mean(
            torch.sum(
                F.binary_cross_entropy_with_logits(reconstruction, data, reduction='none'),
                dim=1
                # dim=(1, 2)
                )  
        )

        # print("labels:", labels)
        # print("labels.shape:", labels.shape)
        # print("labels.dtype:", labels.dtype)
        # print("labels.min():", labels.min().item(), "labels.max():", labels.max().item())
        # print("classification_output.shape:", classification_output.shape)

        classification_loss = F.cross_entropy(classification_output, labels)
                # dim=1
                # dim=(1, 2)
                
        
        kl_loss = -0.5 * torch.sum(1 + z_log_var - z_mean.pow(2) - z_log_var.exp(), dim=1)
        kl_loss = torch.mean(torch.sum(kl_loss))
        total_loss = reconstruction_loss + kl_loss + classification_loss
        total_loss.backward()
        optimizer.step()

        self.total_loss_tracker.append(total_loss.item())
        self.reconstruction_loss_tracker.append(reconstruction_loss.item())
        self.kl_loss_tracker.append(kl_loss.item())
        self.classification_loss_tracker.append(classification_loss.item())
        # print(classification_output.shape, labels.shape)

        preds = torch.softmax(classification_output, dim=1)
        pred_labels = torch.argmax(preds, dim=1)
        correct = (pred_labels == labels).float().sum()
        accuracy = correct / labels.size(0)
        self.accuracy_tracker.append(accuracy.item())

        return {
            "loss": total_loss.item(),
            "reconstruction_loss": reconstruction_loss.item(),
            "kl_loss": kl_loss.item(),
            "classification_loss": classification_loss.item(),
            "accuracy": accuracy.item()
        }

In [20]:
latent_dim = 64
encoded_dim = 512
output_dim = X_train.shape[1]
input_dim = X_train.shape[1]
print(input_dim)


33


In [21]:
class SimpleClassifier(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(SimpleClassifier, self).__init__()
        self.fc_layers = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.2),

            nn.Linear(64, 32),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Dropout(0.1),

            nn.Linear(32, output_dim),
            # nn.Softmax()
        )

    def forward(self, x):
        # print('input: ',x.shape)
        output = self.fc_layers(x)
        # output = output.view(-1)
        # print('output',output.shape)
        return output

In [22]:
classifier = SimpleClassifier(latent_dim, output_dim=2).to(device)


In [23]:
def check_output(model, input_tensor):
    with torch.no_grad():  
        output = model(input_tensor)
        print(f"Input size: {input_tensor.size()}")
        print(f"Output size: {output.size()}")
        print(f"Output: {output}")

model = SimpleClassifier(latent_dim, output_dim=1)

input_tensor = torch.randn(32,latent_dim)  

check_output(model, input_tensor)

Input size: torch.Size([32, 64])
Output size: torch.Size([32, 1])
Output: tensor([[ 0.3256],
        [ 1.0663],
        [-0.0336],
        [-0.2313],
        [ 0.0378],
        [ 0.1005],
        [-0.3622],
        [-0.0750],
        [ 0.7117],
        [ 0.0388],
        [-0.6199],
        [ 0.4649],
        [-0.3589],
        [-0.0025],
        [-0.6826],
        [-0.1581],
        [-0.6116],
        [ 0.1627],
        [ 0.2698],
        [-0.1503],
        [ 0.1860],
        [-0.0651],
        [-0.2239],
        [-0.4741],
        [ 0.1823],
        [ 0.6643],
        [-0.0823],
        [ 0.2351],
        [ 0.0924],
        [-0.1615],
        [-0.1163],
        [-0.6726]])


In [24]:
vae_encoder = VAE_Encoder(latent_dim=latent_dim)

In [25]:
# vae_encoder = VAE_Encoder(latent_dim=latent_dim)
# print("Encoder Summary:")
# # vae_encoder.to(device)

# summary(vae_encoder, input_size=(32, input_dim), device=device)

In [26]:
x = torch.randn(800, X_train.shape[1]).to(device)
steps_output, _ = tabnet_model.encoder(x)
encoded = steps_output[-1]
print(f"Encoded shape: {encoded.shape}")

Encoded shape: torch.Size([800, 512])


In [27]:
import torch

x = torch.randn(800, X_train.shape[1]).to(device)  # Đầu vào có kích thước (batch_size, features)

steps_output, _ = tabnet_model.encoder(x)
print("Shape of encoder output:", [output.shape for output in steps_output])

decoder_input = steps_output[-1]  
decoder_input = decoder_input[None, ...]
try:
    decoder_output = tabnet_model.decoder(decoder_input)
    print(f"Decoder shape: {decoder_output.shape}")
except ValueError as e:
    print(f"Error: {e}")


Shape of encoder output: [torch.Size([800, 512]), torch.Size([800, 512]), torch.Size([800, 512])]
Decoder shape: torch.Size([800, 33])


In [28]:
encoded_dim, output_dim

(512, 33)

In [29]:
vae_decoder = VAE_Decoder(latent_dim=latent_dim, encoded_dim=encoded_dim, output_dim=output_dim).to(device)
print("Decoder Summary:")
summary(vae_decoder, input_size=(32, latent_dim), device=device)

Decoder Summary:


Layer (type:depth-idx)                                       Output Shape              Param #
VAE_Decoder                                                  [32, 33]                  --
├─Sequential: 1-1                                            [32, 512]                 --
│    └─Linear: 2-1                                           [32, 32]                  2,080
│    └─ReLU: 2-2                                             [32, 32]                  --
│    └─Linear: 2-3                                           [32, 96]                  3,168
│    └─ReLU: 2-4                                             [32, 96]                  --
│    └─Linear: 2-5                                           [32, 96]                  9,312
│    └─ReLU: 2-6                                             [32, 96]                  --
│    └─Linear: 2-7                                           [32, 128]                 12,416
│    └─ReLU: 2-8                                             [32, 128]            

In [30]:
vae = VAE_Tabnet_MLPS(encoder=vae_encoder, decoder=vae_decoder,classifier=classifier).to(device)
# summary(vae, input_size=(32, input_dim), device=device)

In [31]:
# learning_rate = 0.0001
# optimizer = optim.Adam(vae.parameters(), lr=learning_rate)
# num_epochs = 10

# for epoch in range(num_epochs):
#     vae.train()
#     train_loss = 0
#     rec_loss = 0
#     kl_loss = 0
#     classification_loss = 0
#     accuracy = 0

#     for batch_data, batch_labels in train_loader:
#         batch_data = batch_data.to(device)
#         batch_labels = batch_labels.to(device)
#         # print(f"Batch data shape: {batch_data.shape}, Batch labels shape: {batch_labels.shape}")
#         results = vae.train_step(batch_data, batch_labels, optimizer)
        
#         train_loss += results["loss"]
#         rec_loss += results["reconstruction_loss"]
#         kl_loss += results["kl_loss"]
#         classification_loss += results["classification_loss"]
#         accuracy += results["accuracy"]

#     train_loss /= len(train_loader)
#     rec_loss /= len(train_loader)
#     kl_loss /= len(train_loader)
#     classification_loss /= len(train_loader)
#     accuracy /= len(train_loader)

#     print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {train_loss:.4f}, Reconstruction Loss: {rec_loss:.4f}, KL Loss: {kl_loss:.4f}, Classification Loss: {classification_loss:.4f}, Accuracy: {accuracy:.4f}")


In [32]:
for param in vae.encoder.parameters():
    param.requires_grad = False

In [33]:
learning_rate = 0.0001
optimizer = optim.Adam(vae.parameters(), lr=learning_rate)
num_epochs = 50
vae_new = VAE_Tabnet_MLPS(vae.encoder, vae.decoder, vae.classifier).to(device)
for param in vae_new.encoder.parameters():
    param.requires_grad = False

optimizer = optim.Adam(filter(lambda p: p.requires_grad, vae_new.parameters()), lr=learning_rate)
for epoch in range(num_epochs):
    vae_new.train()
    train_loss = 0
    rec_loss = 0
    kl_loss = 0
    classification_loss = 0
    accuracy = 0

    for batch_data, batch_labels in train_loader:
        batch_data = batch_data.to(device)
        batch_labels = batch_labels.to(device)
        results = vae.train_step(batch_data, batch_labels, optimizer)
        
        train_loss += results["loss"]
        rec_loss += results["reconstruction_loss"]
        kl_loss += results["kl_loss"]
        classification_loss += results["classification_loss"]
        accuracy += results["accuracy"]

    train_loss /= len(train_loader)
    rec_loss /= len(train_loader)
    kl_loss /= len(train_loader)
    classification_loss /= len(train_loader)
    accuracy /= len(train_loader)

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {train_loss:.4f}, Reconstruction Loss: {rec_loss:.4f}, KL Loss: {kl_loss:.4f}, Classification Loss: {classification_loss:.4f}, Accuracy: {accuracy:.4f}")

Epoch 1/50, Loss: -402.6622, Reconstruction Loss: -434.8605, KL Loss: 31.5445, Classification Loss: 0.6538, Accuracy: 0.6138
Epoch 2/50, Loss: -408.1541, Reconstruction Loss: -440.0485, KL Loss: 31.5452, Classification Loss: 0.3493, Accuracy: 0.8981
Epoch 3/50, Loss: -407.1337, Reconstruction Loss: -438.8141, KL Loss: 31.5459, Classification Loss: 0.1345, Accuracy: 0.9818
Epoch 4/50, Loss: -408.5418, Reconstruction Loss: -440.1528, KL Loss: 31.5452, Classification Loss: 0.0659, Accuracy: 0.9942
Epoch 5/50, Loss: -406.6109, Reconstruction Loss: -438.1958, KL Loss: 31.5440, Classification Loss: 0.0409, Accuracy: 0.9955
Epoch 6/50, Loss: -407.6235, Reconstruction Loss: -439.1914, KL Loss: 31.5441, Classification Loss: 0.0237, Accuracy: 0.9984
Epoch 7/50, Loss: -437.6828, Reconstruction Loss: -469.2460, KL Loss: 31.5452, Classification Loss: 0.0180, Accuracy: 0.9978
Epoch 8/50, Loss: -466.4237, Reconstruction Loss: -497.9810, KL Loss: 31.5447, Classification Loss: 0.0126, Accuracy: 0.9989


In [34]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SimpleDiffusionModel(nn.Module):
    def __init__(self, latent_dim, time_steps=1000):
        super().__init__()
        self.time_steps = time_steps
        self.latent_dim = latent_dim
        
        # Tạo các beta_schedule tuyến tính
        beta = torch.linspace(0.0001, 0.02, time_steps)
        alpha = 1. - beta
        alpha_bar = torch.cumprod(alpha, dim=0)

        self.register_buffer('beta', beta)
        self.register_buffer('alpha', alpha)
        self.register_buffer('alpha_bar', alpha_bar)

        # Mạng neural đơn giản để dự đoán nhiễu
        self.model = nn.Sequential(
            nn.Linear(latent_dim + 1, 512),
            nn.ReLU(),
            nn.Linear(512, latent_dim)
        )

    def forward(self, z, t):
        noise = torch.randn_like(z)
        
        # Đảm bảo t là long type và shape phù hợp
        if isinstance(t, torch.Tensor):
            t = t.to(dtype=torch.long)
        else:
            t = torch.tensor([t], device=z.device, dtype=torch.long).expand(z.shape[0])

        sqrt_alpha_bar = torch.sqrt(self.alpha_bar[t])[:, None]
        sqrt_one_minus_alpha_bar = torch.sqrt(1 - self.alpha_bar[t])[:, None]
        noisy_z = sqrt_alpha_bar * z + sqrt_one_minus_alpha_bar * noise

        predicted_noise = self.model(torch.cat([noisy_z, t.unsqueeze(1)], dim=1))
        loss = F.mse_loss(predicted_noise, noise)
        return loss

    def sample(self, z):
        for i in reversed(range(self.time_steps)):
            z = self.denoise_step(z, i)
        return z
    
    def denoise_step(self, z, t):
        timestep = t.item() if isinstance(t, torch.Tensor) else t
        t_batch = torch.full((z.shape[0],), timestep, device=z.device, dtype=torch.long)

        predicted_noise = self.model(torch.cat([z, t_batch.unsqueeze(1)], dim=1))

        alpha = self.alpha[timestep]
        alpha_bar = self.alpha_bar[timestep]
        beta = self.beta[timestep]

        z = (1 / torch.sqrt(alpha)) * (z - ((1 - alpha) / torch.sqrt(1 - alpha_bar)) * predicted_noise)
        if timestep > 0:
            noise = torch.randn_like(z)
            z += torch.sqrt(beta) * noise
        return z

In [35]:

with torch.no_grad():
    vae_new.encoder.eval()
    dummy_input = torch.randn(1, input_dim).to(device)  # Thay input_dim theo đúng dữ liệu của bạn
    dummy_labels = torch.zeros(1, dtype=torch.long).to(device)
    z_mean, z_log_var, _ = vae_new.encoder(dummy_input, dummy_labels)
    latent_dim = z_mean.shape[1]

In [36]:
# latent_dim = vae.encoder[-1].out_features  # Kích thước latent z
diffusion_model = SimpleDiffusionModel(latent_dim=latent_dim).to(device)
diffusion_optimizer = torch.optim.Adam(diffusion_model.parameters(), lr=1e-3)

In [37]:
class FullPipelineModel(nn.Module):
    def __init__(self, vae, diffusion, classifier):
        super(FullPipelineModel, self).__init__()
        self.vae = vae
        self.diffusion = diffusion
        self.classifier = classifier

        self.total_loss = []
        self.recon_loss = []
        self.kl_loss = []
        self.cls_loss = []
        self.accuracy = []

    def forward(self, x, labels):
        z_mean, z_log_var, z = self.vae.encoder(x, labels)
        reconstruction = self.vae.decoder(z)
        z_diffused = self.diffusion.sample(z)  

        classification_logits = self.classifier(z_diffused).float()
        return reconstruction, z_mean, z_log_var, classification_logits

    def train_step(self, x, labels, optimizer):
        # print(f"Shape of x: {x.shape}, Shape of labels: {labels.shape}")
        if labels.dim() == 2 and labels.size(1) == 2:
            labels = torch.argmax(labels, dim=1)
        # print(f"Shape of x: {x.shape}, Shape of labels: {labels.shape}")
        # labels = labels.squeeze().long().to(device)
        labels = labels.squeeze().to(device)
        optimizer.zero_grad()

        reconstruction, z_mean, z_log_var, classification_logits = self.forward(x, labels)
        recon_loss = F.binary_cross_entropy_with_logits(reconstruction, x, reduction='mean')
        kl_loss = -0.5 * torch.mean(torch.sum(1 + z_log_var - z_mean.pow(2) - z_log_var.exp(), dim=1))
        # print(f'type of classification_logits: {classification_logits.dtype}')
        # print(f'type of labels: {labels.dtype}')
        # print("classification_logits.shape:", classification_logits.shape)
        # print("labels.shape:", labels.shape)
        cls_loss = F.cross_entropy(classification_logits, labels.float())
        total_loss = recon_loss + kl_loss + cls_loss
        total_loss.backward()
        optimizer.step()

        self.total_loss.append(total_loss.item())
        self.recon_loss.append(recon_loss.item())
        self.kl_loss.append(kl_loss.item())
        self.cls_loss.append(cls_loss.item())

        preds = torch.argmax(classification_logits, dim=1)
        labels = torch.argmax(labels, dim=1)
        # print(f'preds: {preds.shape} - labels: {labels.shape}')
        acc = (preds == labels).float().mean()
        self.accuracy.append(acc.item())

        return {
            "total_loss": total_loss.item(),
            "recon_loss": recon_loss.item(),
            "kl_loss": kl_loss.item(),
            "cls_loss": cls_loss.item(),
            "accuracy": acc.item()
        }

In [38]:
from tqdm import tqdm
def train_diffusion(vae, diffusion_model, dataloader, optimizer, device, time_steps=1000, epochs=150):
    diffusion_model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch_data, batch_labels in tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}"):
            batch_data = batch_data.to(device)
            batch_labels = batch_labels.squeeze().to(device)
            if batch_labels.dim() > 1:  # nếu là one-hot hoặc có chiều phụ
                batch_labels = batch_labels.argmax(dim=1)
            with torch.no_grad():
                z_mean, z_log_var, z = vae.encoder(batch_data, batch_labels)

            t = torch.randint(0, time_steps, (z.shape[0],), device=device).long()
            loss = diffusion_model(z, t)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {epoch+1}, Diffusion Loss: {total_loss/len(dataloader):.4f}")

In [39]:
from sklearn.metrics import classification_report, confusion_matrix, f1_score, precision_score, recall_score

def evaluate_diffusion_with_classifier(vae, diffusion_model, classifier, test_loader, device, time_steps=1000):
    vae.eval()
    diffusion_model.eval()
    classifier.eval()
    
    correct = 0
    total = 0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for batch_data, batch_labels in test_loader:
            batch_data = batch_data.to(device)
            batch_labels = batch_labels.squeeze().to(device)
            if batch_labels.ndim == 3 and batch_labels.shape[1] == 1:
                batch_labels = batch_labels.squeeze(1)  # bỏ chiều 1

            if batch_labels.ndim == 2 and batch_labels.shape[1] == 2:
                batch_labels = torch.argmax(batch_labels, dim=1)
            elif batch_labels.ndim == 2 and batch_labels.shape[1] == 1:
                batch_labels = batch_labels.squeeze(1)

            z_mean, z_log_var, z = vae.encoder(batch_data, batch_labels)

            t_forward = time_steps - 1
            sqrt_alpha_bar = torch.sqrt(diffusion_model.alpha_bar[t_forward])
            sqrt_one_minus_alpha_bar = torch.sqrt(1 - diffusion_model.alpha_bar[t_forward])
            noisy_z = sqrt_alpha_bar * z + sqrt_one_minus_alpha_bar * torch.randn_like(z)

            z_recovered = diffusion_model.sample(noisy_z)
            logits = classifier(z_recovered)

            # print("Logits shape:", logits.shape)  # Debug

            if len(logits.shape) == 1:
                preds = (torch.sigmoid(logits) > 0.5).float()
            elif len(logits.shape) == 2:
                preds = torch.argmax(logits, dim=1)
            else:
                raise ValueError(f"Unexpected logits shape: {logits.shape}")

            # print(f"preds shape: {preds.shape}, batch_labels shape: {batch_labels.shape}")  # Debug
            preds = preds.cpu().numpy().flatten()
            labels = batch_labels.squeeze().cpu().numpy().flatten()

            # Lưu lại cho metrics
            all_preds.extend(preds)
            all_labels.extend(labels)

            correct += (preds == labels).sum()
            total += len(labels)

    accuracy = correct / total
    print("\nClassification Report:")
    print(classification_report(all_labels, all_preds))

    print("\nConfusion Matrix:")
    print(confusion_matrix(all_labels, all_preds))

    # Nếu bạn muốn trả về thêm các chỉ số cụ thể:
    accuracy = sum(np.array(all_preds) == np.array(all_labels)) / len(all_labels)
    f1 = f1_score(all_labels, all_preds, average='binary' if len(np.unique(all_labels)) == 2 else 'macro')
    precision = precision_score(all_labels, all_preds, average='binary' if len(np.unique(all_labels)) == 2 else 'macro')
    recall = recall_score(all_labels, all_preds, average='binary' if len(np.unique(all_labels)) == 2 else 'macro')

    return {
        "accuracy": accuracy,
        "f1_score": f1,
        "precision": precision,
        "recall": recall
    }
    print(f"Accuracy on recovered z: {accuracy:.4f}")


In [40]:
device

device(type='cuda')

In [41]:
# latent_dim = vae.encoder[-1].out_features
diffusion_model = SimpleDiffusionModel(latent_dim=latent_dim).to(device)
diffusion_optimizer = optim.Adam(diffusion_model.parameters(), lr=1e-3)

In [ ]:
# Giả sử bạn đã có sẵn các model con
full_model = FullPipelineModel(vae=vae, diffusion=diffusion_model, classifier=classifier).to(device)
optimizer = torch.optim.Adam(full_model.parameters(), lr=1e-4)

num_epochs = 50
for epoch in range(num_epochs):
    full_model.train()
    for batch_data, batch_labels in train_loader:
        batch_data = batch_data.to(device)
        batch_labels = batch_labels.to(device)

        # Nếu label là one-hot encoded, chuyển thành dạng long
        if batch_labels.dim() == 2 and batch_labels.size(1) == 2:
            batch_labels = torch.argmax(batch_labels, dim=1)

        metrics = full_model.train_step(batch_data, batch_labels, optimizer)

    print(f"Epoch {epoch+1} | Loss: {metrics['total_loss']:.4f} | Acc: {metrics['accuracy']:.4f}")

metrics = evaluate_diffusion_with_classifier(
    vae=vae,
    diffusion_model=diffusion_model,
    classifier=classifier,
    test_loader=test_loader,
    device=device,
    time_steps=100
)

print(f"Accuracy on recovered z: {metrics['accuracy']:.4f}")
print(f"F1 Score: {metrics['f1_score']:.4f}")
print(f"Precision: {metrics['precision']:.4f}")
print(f"Recall: {metrics['recall']:.4f}")

Epoch 1 | Loss: -15.7032 | Acc: 0.7632
Epoch 2 | Loss: -13.0239 | Acc: 0.6842


In [ ]:
# # latent_dim = vae.encoder[-1].out_features
# diffusion_model = SimpleDiffusionModel(latent_dim=latent_dim).to(device)
# diffusion_optimizer = optim.Adam(diffusion_model.parameters(), lr=1e-3)

# train_diffusion(vae_new, diffusion_model, train_loader, diffusion_optimizer, device, epochs=100)

# evaluate_diffusion_with_classifier(vae_new, diffusion_model, vae.classifier, test_loader, device)